In [1]:
#1.데이터 로드
import pandas as pd

df = pd.read_csv('data/train.csv')
#print(df.head())

In [2]:
#2.불필요 컬럼 삭제

#print(df.info())
#print(df.select_dtypes('object').nunique())
#print(df['Cabin'].value_counts())
#sns.countplot(data=df, x='Cabin')

df = df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [3]:
#3.결측치 처리

#print(df.info())
#print(df['Embarked'].value_counts())
#sns.countplot(data=df, x='Embarked')

df['Embarked'] = df['Embarked'].fillna('S') #Embarked 탑승지역은 S가 압도적으로 많고, 결측치 수가 적으므로 S로 대체
df = df.dropna() #평균이나 중앙값으로 대체하기에는 Age가 한쪽으로 몰리게 되므로, 결측치 제거 

In [4]:
'''
def graph(variable):
    survived = df[df['Survived']==1][variable].value_counts()
    dead = df[df['Survived']==0][variable].value_counts()
    result = pd.DataFrame([survived,dead])
    result.index = ['Survived','Dead']
    result.T.plot(kind='bar', color=['skyblue','gray'], rot=0, figsize=(8,6))
'''

"\ndef graph(variable):\n    survived = df[df['Survived']==1][variable].value_counts()\n    dead = df[df['Survived']==0][variable].value_counts()\n    result = pd.DataFrame([survived,dead])\n    result.index = ['Survived','Dead']\n    result.T.plot(kind='bar', color=['skyblue','gray'], rot=0, figsize=(8,6))\n"

In [5]:
#4.인코딩
df = pd.get_dummies(df,columns=['Sex'])
df = pd.get_dummies(df,columns=['Embarked'])
df = pd.get_dummies(df,columns=['Pclass'])

df['Sex_female'] = df['Sex_female'].astype('int')
df['Sex_male'] = df['Sex_male'].astype('int')
df['Embarked_C'] = df['Embarked_C'].astype('int')
df['Embarked_Q'] = df['Embarked_Q'].astype('int')
df['Embarked_S'] = df['Embarked_S'].astype('int')
df['Pclass_1'] = df['Pclass_1'].astype('int')
df['Pclass_2'] = df['Pclass_2'].astype('int')
df['Pclass_3'] = df['Pclass_3'].astype('int')

#df.info()
#df = df.drop(columns=['Sex_male','Embarked_Q'])

In [6]:
#5.모델에 학습할 독립변수 추출
import statsmodels.api as sm

x = df[['Age','SibSp','Parch','Fare','Sex_female','Embarked_C','Embarked_Q','Pclass_2','Pclass_3']]
y = df['Survived']

model = sm.Logit(y,x)
model_fit = model.fit()
print(model_fit.summary())

Optimization terminated successfully.
         Current function value: 0.449733
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      705
Method:                           MLE   Df Model:                            8
Date:                Thu, 23 May 2024   Pseudo R-squ.:                  0.3342
Time:                        14:13:07   Log-Likelihood:                -321.11
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.340e-65
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age           -0.0239      0.005     -4.675      0.000      -0.034      -0.014
SibSp         -0.2810      0.

In [7]:
#6.데이터 분리하기
x = df[['Age','SibSp','Fare','Sex_female','Embarked_C','Pclass_3']]
y = df['Survived']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7,test_size=0.3,random_state=0)

In [8]:
#7.모델 학습(train)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=999)
model.fit(x_train,y_train)

LogisticRegression()

In [9]:
#8.모델 테스트(test) 및 평가
from sklearn.metrics import accuracy_score, confusion_matrix

y_predict = model.predict(x_test)
print(accuracy_score(y_predict, y_test))

0.8186046511627907
